In [59]:
! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
import setup_google_colab

--2020-05-20 12:14:42--  https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1991 (1.9K) [text/plain]
Saving to: ‘setup_google_colab.py’

setup_google_colab. 100%[===================>]   1.94K  --.-KB/s    in 0s      

2020-05-20 12:14:42 (29.3 MB/s) - ‘setup_google_colab.py’ saved [1991/1991]



In [0]:
setup_google_colab.setup_project()

In [61]:
!git clone -l -s https://github.com/sumanthkv-bmsce/ChatFinal.git cloned_repo
%cd cloned_repo
!ls

Cloning into 'cloned_repo'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 61 (delta 13), reused 54 (delta 7), pack-reused 0
Unpacking objects: 100% (61/61), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned_repo/cloned_repo/cloned_repo
AWS-tutorial.md  Docker-tutorial.md  README.md		    week2
common		 honor		     setup_google_colab.py  week3
docker		 project	     week1		    week4


In [69]:
import sys
sys.path.append("..")
from common.download_utils import download_project_resources
from common.download_utils import download_week3_resources

download_project_resources()
download_week3_resources()


File data/dialogues.tsv is already downloaded.
File data/tagged_posts.tsv is already downloaded.


In [0]:
from project.utils import *

In [0]:
def load_embeddings1(embeddings_path):
    """Loads pre-trained word embeddings from tsv file.
    Args:
      embeddings_path - path to the embeddings file.
    Returns:
      embeddings - dict mapping words to vectors;
      embeddings_dim - dimension of the vectors.
    """

    # Hint: you have already implemented a similar routine in the 3rd assignment.
    # Note that here you also need to know the dimension of the loaded embeddings.
    # When you load the embeddings, use numpy.float32 type as dtype

    ########################
    #### YOUR CODE HERE ####
    ########################


    embeddings = dict()
    for line in open(embeddings_path, encoding='utf-8'):
        row = line.strip().split('\t')
        embeddings[row[0]] = np.array(row[1:], dtype=np.float32)
    embeddings_dim = embeddings[list(embeddings)[0]].shape[0]
    
    return embeddings, embeddings_dim

In [0]:
import gensim
wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000)

300

In [0]:
import pandas as pd
import pickle
import re

from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:

def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
    
    # Train a vectorizer on X_train data.
    # Transform X_train and X_test data.
    
    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    tf_id_vector = TfidfVectorizer(ngram_range=(1,2),max_df=0.9,min_df=5,token_pattern='(\S+)')
    X_train = tf_id_vector.fit_transform(X_train)
    X_test = tf_id_vector.transform(X_test)

    with open(vectorizer_path, 'wb') as f:
        pickle.dump(tf_id_vector, f)
    
    return X_train, X_test

In [0]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogues.tsv', sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/tagged_posts.tsv', sep='\t').sample(sample_size, random_state=0)

In [0]:
dialogue_df.head()

,text,tag
82925,"Donna, you are a muffin.",dialogue
48774,He was here last night till about two o'clock....,dialogue
55394,"All right, then make an appointment with her s...",dialogue
90806,"Hey, what is this-an interview? We're supposed...",dialogue
107758,Yeah. He's just a friend of mine I was trying ...,dialogue


In [0]:
stackoverflow_df

,post_id,title,tag
2168983,43837842,Efficient Algorithm to compose valid expressio...,python
1084095,15747223,Why does this basic thread program fail with C...,c_cpp
1049020,15189594,Link to scroll to top not working,javascript
200466,3273927,Is it possible to implement ping on windows ph...,c#
1200249,17684551,GLSL normal mapping issue,c_cpp
...,...,...,...
189987,3128254,"Cakephp, ordering associated tables",php
1975274,33828988,setProperty value from requested getParameter,java
445525,6519634,"How can I send some HTML, render a view, then ...",javascript
187718,3094875,What's the standard way of trapping system mes...,c_cpp


In [0]:

from project.utils import text_prepare

In [0]:
dialogue_df['text'] = dialogue_df['text'].apply(lambda x: text_prepare(x))
stackoverflow_df['title'] = stackoverflow_df["title"].apply(lambda x: text_prepare(x))

In [0]:

from sklearn.model_selection import train_test_split

In [0]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test, RESOURCE_PATH['TFIDF_VECTORIZER'])

Train size = 360000, test size = 40000


In [0]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [0]:
log_reg = LogisticRegression(penalty='l2',C=10,random_state=0)
log_reg.fit(X_train_tfidf,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:

# Check test accuracy.
y_test_pred = log_reg.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.9916


In [0]:
pickle.dump(log_reg, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

In [0]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


In [0]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train), vectorizer.transform(X_test)

In [0]:

from sklearn.multiclass import OneVsRestClassifier

In [0]:
tag_classifier = OneVsRestClassifier(LogisticRegression(penalty='l2',C=5,random_state=0,max_iter=500))
tag_classifier.fit(X_train_tfidf,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=5, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=500,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=0, solver='lbfgs',
                                                 tol=0.0001, verbose=0,
                                                 warm_start=False),
                    n_jobs=None)

In [0]:
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.8007


In [0]:

pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

In [0]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

In [79]:

posts_df.head()

,post_id,title,tag
0,9,Calculate age in C#,c#
1,16,Filling a DataSet or DataTable from a LINQ que...,c#
2,39,Reliable timer in a console application,c#
3,42,Best way to allow plugins for a PHP application,php
4,59,"How do I get a distinct, ordered list of names...",c#


In [0]:
counts_by_tag = posts_df.groupby(['tag'])['tag'].count()

In [81]:

list(counts_by_tag.items())

[('c#', 394451),
 ('c_cpp', 281300),
 ('java', 383456),
 ('javascript', 375867),
 ('php', 321752),
 ('python', 208607),
 ('r', 36359),
 ('ruby', 99930),
 ('swift', 34809),
 ('vb', 35044)]

In [0]:
def question_to_vec(question, embeddings, dim):
    """Transforms a string to an embedding by averaging word embeddings."""
    
    # Hint: you have already implemented exactly this function in the 3rd assignment.

    ########################
    #### YOUR CODE HERE ####
    ########################
    result = np.zeros(dim)
    cnt = 0
    words = question.split()
    for word in words:
        if word in embeddings:
            result += np.array(embeddings[word])
            cnt += 1
    if cnt != 0:
        result /= cnt
    return result 

In [0]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in counts_by_tag.items():
    tag_posts = posts_df[posts_df['tag'] == tag]
    
    tag_post_ids = tag_posts['post_id'].values ######### YOUR CODE HERE #############
    
    tag_vectors = np.zeros((count, 300), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(title, wv_embeddings, 300) ######### YOUR CODE HERE #############

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))